In [ ]:
from credentials import user, password, rds_host
import pymysql
import pandas as pd
from boston_functions import *
import re
import numpy as np
from liquid import liquids
from garnish import garnishes
import random

In [ ]:
data = pd.read_csv("./mr-boston-all-glasses.csv")

data = data[data.loc[:, "glass-size"].notna()]

valid_units = ["oz", "tsp", "splash", "dash"]

fill_liquid = ["ginger ale", "carbonated water", "cola", "water", "chilled champagne", "soda water", 
               "club soda", "ginger ale or soda water", "lemon-lime soda", "ginger beer", "bitter lemon soda",
               "apple juice", "orange juice"]

invaild_ingredients = ['chopped', 'cut in half', 'cut in halves', 'cut into halves', 'flamed', 'hulled', 'long', 'skinned',
                       'whipped', "preferably b.a. reynold's", 'preferably jamaican', 'preferably japanese', 
                       'preferably pedro ximenez', "such as bittermen's elemakule", 'such as demerara', 
                       'such as islay or skye', 'such as nasturtium']


In [ ]:
ingredient_indicies = range(3, 14)

all_recipies, measures = get_cocktail_recipies(data, ingredient_indicies, liquids, garnishes, invaild_ingredients, valid_units)

categories = list(set(data.iloc[:, 1])) + ["AI Instant Classic"]

liquid_colors = pd.read_csv("./LIquid_Colors_Final.csv")


# populate_liquid_table(liquid_colors)

In [ ]:
def populate_liquid_table(liquid_df):
    conn = pymysql.connect(rds_host, user=user, password=password, connect_timeout=50)
    cursor = conn.cursor()
    cursor.execute('USE cocktailproject')
    for row in range(len(liquid_df)):
        liquid = liquid_df.iloc[row, 0]
        hex_color = liquid_df.iloc[row, 1]
        print(liquid, hex_color)
        sql = f"INSERT INTO Liquids (Liquid_Name, Color) VALUES ('{liquid}', '{hex_color}');"
        cursor.execute(sql)
    conn.commit()
    cursor.execute("SELECT * FROM Liquids")
    data = cursor.fetchall()
    print(data)
    conn.close()


# populate_garnish_table(garnishes)

In [ ]:
def populate_garnish_table(garnishes):
    conn = pymysql.connect(rds_host, user=user, password=password, connect_timeout=50)
    cursor = conn.cursor()
    cursor.execute('USE cocktailproject')
    for garnish in garnishes:
        sql = f"INSERT INTO Garnishes (Garnish_Name) VALUES ('{garnish}');"
        cursor.execute(sql)
    conn.commit()
    cursor.execute("SELECT * FROM Garnishes")
    data = cursor.fetchall()
    print(data)
    conn.close()


# populate_category_table(categories)

In [ ]:
def populate_category_table(categories):
    conn = pymysql.connect(rds_host, user=user, password=password, connect_timeout=50)
    cursor = conn.cursor()
    cursor.execute('USE cocktailproject')
    for category in categories:
        sql = f"INSERT INTO Categories (Category_Name) VALUES ('{category}');"
        cursor.execute(sql)
    conn.commit()
    cursor.execute("SELECT * FROM Categories")
    data = cursor.fetchall()
    print(data)
    conn.close()

# populate_glass_table(Glasses)

In [ ]:
#set empty lists for cocktail names, glass names, and glass sizes
names = []
glasses = []
glass_sizes = []

# locate cocktail names, glass names, and glass sizes
for i in range(len(all_recipies)):
    names.append(all_recipies[i]["name"])
    glasses.append(all_recipies[i]["glass"])
    glass_sizes.append(all_recipies[i]["glass_size"])
#create df
glass_df = pd.DataFrame({"cocktail_name": names, "glass_name": glasses, "glass_size": glass_sizes})

# glass_df.head()

In [ ]:
#read glass svg csv
svg_data = pd.read_csv("../glasses.csv")
svg_data.head()

In [ ]:
def populate_glass_table(glass_df):
    conn = pymysql.connect(rds_host, user=user, password=password, connect_timeout=50)
    cursor = conn.cursor()
    cursor.execute('USE cocktailproject')
    for row in range(len(glass_df)):
        glass_name = glass_df.iloc[row, 1]
        glass_size = glass_df.iloc[row, 2]
        mask = glass_df.iloc[row, 3]
        path = glass_df.iloc[row, 4]
        mask_height = glass_df.iloc[row, 5]
        mask_top_margin = glass_df.iloc[row, 6]
        print(glass_name, glass_size)
        sql = f"INSERT INTO Glasses (Glass_Name, Glass_Size, Mask, Path, Mask_Height, Mask_Top_Margin) VALUES ('{glass_name}', '{glass_size}', '{mask}', '{path}', '{mask_height}', '{mask_top_margin}');"
        cursor.execute(sql)
    conn.commit()
    cursor.execute("SELECT * FROM Glasses")
    data = cursor.fetchall()
    print(data)
    conn.close()

# populate_cocktail_table(cocktail_df)

In [ ]:
#set empty lists for instructions, category and glass ids
category_ids = []
glass_ids = []
instructions_list = []

#connect to sql
conn = pymysql.connect(rds_host, user=user, password=password, connect_timeout=50)
#create cursor object
cursor = conn.cursor()
#use cocktail db
cursor.execute('USE cocktailproject')

#for each recipe,
for i in range(len(all_recipies)):
    #find instructions and append to instructions list
    instructions = all_recipies[i]["instructions"]
    instructions_list.append(instructions)
    
    #find cocktail category
    category = all_recipies[i]["category"]
    
    #select category id and append to category id list
    sql = f"SELECT Category_ID FROM Categories WHERE Category_Name='{category}'"
    cursor.execute(sql)
    data = cursor.fetchall()
    category_ids.append(data[0][0])
    
    #select glass id and append to glass id list
    sql = f"SELECT Glass_ID FROM Glasses WHERE Glass_Name='{glasses[i]}'"
    cursor.execute(sql)
    data = cursor.fetchall()
    glass_ids.append(data[0][0])

#close sql connection
conn.close()

In [ ]:
def populate_cocktail_table(cocktail_df):
    conn = pymysql.connect(rds_host, user=user, password=password, connect_timeout=50)
    cursor = conn.cursor()
    cursor.execute('USE cocktailproject')
    for row in range(len(cocktail_df)):
        cocktail_name = cocktail_df.iloc[row, 0]
        glass_id = cocktail_df.iloc[row, 1]
        category_id = cocktail_df.iloc[row, 2]
        instructions = cocktail_df.iloc[row, 3]
        print(cocktail_name)
        sql = f"INSERT INTO Cocktails (Cocktail_Name, Glass_ID, Category_ID, Instructions) VALUES ('{cocktail_name}', '{glass_id}', '{category_id}', '{instructions}');"
        cursor.execute(sql)
    conn.commit()
    cursor.execute("SELECT * FROM Cocktails")
    data = cursor.fetchall()
    print(data)
    conn.close()
# populate_cocktail_table(cocktail_df)

# populate_liquid_instructions_table(liquid_ingredients_df)

In [ ]:
liquid_instructions_table_sql = "CREATE TABLE Liquid_Instuctions ( \
                                Liquid_Instruction_ID INT NOT NULL PRIMARY KEY AUTO_INCREMENT \
                                , Cocktail_ID INT \
                                , Liquid_ID INT \
                                , Measure VARCHAR(24) \
                                , FOREIGN KEY (Cocktail_ID) REFERENCES Cocktails(Cocktail_ID) \
                                , FOREIGN KEY (Liquid_ID) REFERENCES Liquids(Liquid_ID) \
                                );"
all_recipies[2]

In [ ]:
#set empty lists for liquid and cocktail ids, liquid ingredients and measures
cocktail_ids_list = []
liquid_ingredient_list = []
liquid_measure_list = []
liquid_ids = []

#for each recipe,
for i in range(len(all_recipies)):
    #set cocktail_id
    cocktail_id = cocktail_id_list[i]
    #find ingredient list
    ingredient_list = all_recipies[i]["recipe"]
    #for each non-garnish ingredient,
    for measure, ingredient in ingredient_list:
        if measure == "add":
            pass
        elif ingredient == "pimm's cup":
            pass
        #add to lists
        else:
            cocktail_ids_list.append(cocktail_id)
            liquid_measure_list.append(measure)
            liquid_ingredient_list.append(ingredient)
            
#connect to sql
conn = pymysql.connect(rds_host, user=user, password=password, connect_timeout=50)
#create cursor object
cursor = conn.cursor()
#use cocktail db
cursor.execute('USE cocktailproject')


#for each ingredient,
for ingredient in liquid_ingredient_list:
    print(ingredient)
    if ingredient[:6] == "rose's":
        ingredient = ingredient[7:]
        print(ingredient)
        #select liquid id and append to liquid id list
        sql = f'SELECT Liquid_ID FROM Liquids WHERE Liquid_Name="{ingredient}"'
        cursor.execute(sql)
        data = cursor.fetchall()
        print(data[0][0])
        liquid_ids.append(data[0][0])
    elif ingredient[:8] =="peychaud":
        ingredient = "peychauds bitters"
        print(ingredient)
        #select liquid id and append to liquid id list
        sql = f'SELECT Liquid_ID FROM Liquids WHERE Liquid_Name="{ingredient}"'
        cursor.execute(sql)
        data = cursor.fetchall()
        print(data[0][0])
        liquid_ids.append(data[0][0])
    else:
        #select liquid id and append to liquid id list
        sql = f'SELECT Liquid_ID FROM Liquids WHERE Liquid_Name="{ingredient}"'
        cursor.execute(sql)
        data = cursor.fetchall()
        print(data[0][0])
        liquid_ids.append(data[0][0])

#close sql connection
conn.close()
    
liquid_ingredients_df = pd.DataFrame({"cocktail_id": cocktail_ids_list, "liquid_id": liquid_ids, 
                                      "measure": liquid_measure_list})

liquid_ingredients_df.head()


In [ ]:
print(len(cocktail_ids_list), len(liquid_ids), len(liquid_measure_list))

In [ ]:
def populate_liquid_instructions_table(liquid_instructions_df):
    conn = pymysql.connect(rds_host, user=user, password=password, connect_timeout=50)
    cursor = conn.cursor()
    cursor.execute('USE cocktailproject')
    for row in range(len(liquid_instructions_df)):
        cocktail_id = liquid_instructions_df.iloc[row, 0]
        liquid_id = liquid_instructions_df.iloc[row, 1]
        measure = liquid_instructions_df.iloc[row, 2]
        print(liquid_id, measure)
        sql = f"INSERT INTO Liquid_Instuctions (Cocktail_ID, Liquid_ID, Measure) VALUES ('{cocktail_id}', '{liquid_id}', '{measure}');"
        cursor.execute(sql)
    conn.commit()
    cursor.execute("SELECT * FROM Liquid_Instuctions")
    data = cursor.fetchall()
    print(data)
    conn.close()
# populate_liquid_instructions_table(liquid_ingredients_df)

# populate_garnish_instructions_table(garnish_instructions_df)

In [ ]:
#set empty lists for cocktail ids, liquid ingredients and measures
cocktail_ids_list = []
garnish_ingredient_list = []

#for each recipe,
for i in range(len(all_recipies)):
    #set cocktail_id
    cocktail_id = cocktail_id_list[i]
    #find ingredient list
    ingredient_list = all_recipies[i]["recipe"]
    #for each non-garnish ingredient,
    for measure, ingredient in ingredient_list:
        if measure != "add":
            pass
        else:
            cocktail_ids_list.append(cocktail_id)
            garnish_ingredient_list.append(ingredient)
            
#connect to sql
conn = pymysql.connect(rds_host, user=user, password=password, connect_timeout=50)
#create cursor object
cursor = conn.cursor()
#use cocktail db
cursor.execute('USE cocktailproject')

#set empty list for liquid ids
garnish_ids = []

#for each ingredient,
for ingredient in garnish_ingredient_list:
    print(ingredient)
#     if ingredient[:6] == "rose's":
#         ingredient = ingredient[7:]
#         print(ingredient)
#         #select liquid id and append to liquid id list
#         sql = f'SELECT Liquid_ID FROM Liquids WHERE Liquid_Name="{ingredient}"'
#         cursor.execute(sql)
#         data = cursor.fetchall()
#         print(data[0][0])
#         liquid_ids.append(data[0][0])
#     elif ingredient[:8] =="peychaud":
#         ingredient = "peychauds bitters"
#         print(ingredient)
#         #select liquid id and append to liquid id list
#         sql = f'SELECT Liquid_ID FROM Liquids WHERE Liquid_Name="{ingredient}"'
#         cursor.execute(sql)
#         data = cursor.fetchall()
#         print(data[0][0])
#         liquid_ids.append(data[0][0])
#     else:
    #select liquid id and append to liquid id list
    sql = f'SELECT Garnish_ID FROM Garnishes WHERE Garnish_Name="{ingredient}"'
    cursor.execute(sql)
    data = cursor.fetchall()
    print(data[0][0])
    garnish_ids.append(data[0][0])

#close sql connection
conn.close()
    
garnish_ingredients_df = pd.DataFrame({"cocktail_id": cocktail_ids_list, "garnish_id": garnish_ids})

garnish_ingredients_df.head()

In [ ]:
def populate_garnish_instructions_table(garnish_instructions_df):
    conn = pymysql.connect(rds_host, user=user, password=password, connect_timeout=50)
    cursor = conn.cursor()
    cursor.execute('USE cocktailproject')
    for row in range(len(garnish_instructions_df)):
        cocktail_id = garnish_instructions_df.iloc[row, 0]
        garnish_id = garnish_instructions_df.iloc[row, 1]
        print(garnish_id)
        sql = f"INSERT INTO Garnish_Instuctions (Cocktail_ID, Garnish_ID) VALUES ('{cocktail_id}', '{garnish_id}');"
        cursor.execute(sql)
    conn.commit()
    cursor.execute("SELECT * FROM Garnish_Instuctions")
    data = cursor.fetchall()
    print(data)
    conn.close()
populate_garnish_instructions_table(garnish_ingredients_df)

# populate_rating_table(rating_df):

In [ ]:
rating_table_sql = "CREATE TABLE Ratings ( \
                Rating_ID INT NOT NULL PRIMARY KEY AUTO_INCREMENT \
                , Rating INT \
                , Cocktail_ID INT\
                , FOREIGN KEY (Cocktail_ID) REFERENCES Cocktails(Cocktail_ID) \
                );"

In [ ]:
#connect to sql, create cursor object, and use cocktail db
conn = pymysql.connect(rds_host, user=user, password=password, connect_timeout=50)
cursor = conn.cursor()
cursor.execute('USE cocktailproject')

sql = "SELECT Cocktail_ID FROM Cocktails"
cursor.execute(sql)
data = cursor.fetchall()
cocktail_ids_data = data
cocktail_ids = []
for i in range(len(cocktail_ids_data)):
    
    cocktail_ids.append(cocktail_ids_data[i][0])


conn.close()
len(cocktail_ids)

In [ ]:
cocktail_id_list = []
ratings_list = []
for _id in cocktail_ids:
    for i in range(5):
        cocktail_id_list.append(_id)
        ratings_list.append(random.randint(1,5))

rating_df = pd.DataFrame({"rating": ratings_list, "cocktail_id": cocktail_id_list})

In [ ]:
def populate_rating_table(rating_df):
    conn = pymysql.connect(rds_host, user=user, password=password, connect_timeout=50)
    cursor = conn.cursor()
    cursor.execute('USE cocktailproject')
    for row in range(len(rating_df)):
        rating = rating_df.iloc[row, 0]
        cocktail_id = rating_df.iloc[row, 1]
        print(rating)
        sql = f"INSERT INTO Ratings (Rating, Cocktail_ID) VALUES ('{rating}','{cocktail_id}');"
        cursor.execute(sql)
    conn.commit()
    cursor.execute("SELECT * FROM Ratings")
    data = cursor.fetchall()
    print(data)
    conn.close()
# populate_rating_table(rating_df)

In [9]:
float(1) + float(1/2)

1.5

In [10]:
from fractions import Fraction
float(sum(Fraction(s) for s in '1 2/3'.split()))

1.6666666666666667

In [12]:
"splashes"[:6]

'splash'